In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Make the graphs a bit prettier, and bigger
plt.style.use('ggplot')

# This is necessary to show lots of columns in pandas 0.12. 
# Not necessary in pandas 0.13.
pd.set_option('display.max_colwidth', 5000) 
pd.set_option('display.max_columns', 60)

plt.rcParams['figure.figsize'] = (15, 5)

We're going to use a new dataset here, to demonstrate how to deal with larger datasets. This is a subset of the of 311 service requests from [NYC Open Data](https://nycopendata.socrata.com/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9). 

In [2]:
complaints = pd.read_csv('../data/311-service-requests.csv')

/home/gabriela/.local/share/virtualenvs/pandas-cookbook-M4OY_8hn/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Depending on your pandas version, you might see an error like "DtypeWarning: Columns (8) have mixed types". This means that it's encountered a problem reading in our data. In this case it almost certainly means that it has columns where some of the entries are strings and some are integers.

For now we're going to ignore it and hope we don't run into a problem, but in the long run we'd need to investigate this warning.

# 2.1 What's even in it? (the summary)

When you print a large dataframe, it will only show you the first few rows.

If you don't see this, don't panic! The default behavior for large dataframes changed between pandas 0.12 and 0.13. Previous to 0.13 it would show you a summary of the dataframe. This includes all the columns, and how many non-null values there are in each column.

In [3]:
complaints

Unique Key            Created Date             Closed Date Agency  \
0         26589651  10/31/2013 02:08:41 AM                     NaN   NYPD   
1         26593698  10/31/2013 02:01:04 AM                     NaN   NYPD   
2         26594139  10/31/2013 02:00:24 AM  10/31/2013 02:40:32 AM   NYPD   
3         26595721  10/31/2013 01:56:23 AM  10/31/2013 02:21:48 AM   NYPD   
4         26590930  10/31/2013 01:53:44 AM                     NaN  DOHMH   
5         26592370  10/31/2013 01:46:52 AM                     NaN   NYPD   
6         26595682  10/31/2013 01:46:40 AM                     NaN   NYPD   
7         26595195  10/31/2013 01:44:19 AM  10/31/2013 01:58:49 AM   NYPD   
8         26590540  10/31/2013 01:44:14 AM  10/31/2013 02:28:04 AM   NYPD   
9         26594392  10/31/2013 01:34:41 AM  10/31/2013 02:23:51 AM   NYPD   
10        26595176  10/31/2013 01:25:12 AM                     NaN   NYPD   
11        26591982  10/31/2013 01:24:14 AM  10/31/2013 01:54:39 AM   NYPD   
12        26594169  10/31/2013 01:20:57 AM  10/31/2013 02:12:31 AM   NYPD   
13        26594391  10/31/2013 01:20:13 AM                     NaN   NYPD   
14        26590917  10/31/2013 01:19:54 AM                     NaN  DOHMH   
15        26591458  10/31/2013 01:14:02 AM  10/31/2013 01:30:34 AM   NYPD   
16        26594086  10/31/2013 12:54:03 AM  10/31/2013 02:16:39 AM   NYPD   
17        26595117  10/31/2013 12:52:46 AM                     NaN   NYPD   
18        26590389  10/31/2013 12:51:00 AM                     NaN    DOT   
19        26594210  10/31/2013 12:46:27 AM                     NaN   NYPD   
20        26592932  10/31/2013 12:43:47 AM  10/31/2013 12:56:20 AM   NYPD   
21        26594152  10/31/2013 12:41:17 AM  10/31/2013 01:04:37 AM   NYPD   
22        26589678  10/31/2013 12:39:55 AM                     NaN   NYPD   
23        26592304  10/31/2013 12:38:00 AM                     NaN   NYPD   
24        26591892  10/31/2013 12:37:16 AM                     NaN   NYPD   
25        26591573  10/31/2013 12:35:18 AM  10/31/2013 02:41:35 AM   NYPD   
26        26590509  10/31/2013 12:33:00 AM                     NaN    DOT   
27        26591379  10/31/2013 12:32:44 AM                     NaN  DOHMH   
28        26594085  10/31/2013 12:32:08 AM                     NaN   NYPD   
29        26589201  10/31/2013 12:32:00 AM                     NaN    DOT   
...            ...                     ...                     ...    ...   
111039    26428764  10/04/2013 12:17:03 AM  10/04/2013 12:38:37 AM   NYPD   
111040    26426166  10/04/2013 12:16:22 AM  10/04/2013 05:50:49 AM   NYPD   
111041    26438565  10/04/2013 12:16:00 AM                     NaN    DEP   
111042    26428990  10/04/2013 12:15:52 AM  10/04/2013 12:44:52 AM   NYPD   
111043    26432659  10/04/2013 12:15:46 AM  10/04/2013 04:18:45 AM   NYPD   
111044    26426096  10/04/2013 12:14:09 AM  10/04/2013 01:03:46 AM   NYPD   
111045    26437764  10/04/2013 12:14:00 AM  10/04/2013 12:14:00 AM    DEP   
111046    26436286  10/04/2013 12:14:00 AM                     NaN    DEP   
111047    26428989  10/04/2013 12:13:08 AM  10/04/2013 02:12:47 AM   NYPD   
111048    26430030  10/04/2013 12:12:07 AM  10/04/2013 02:45:24 AM   NYPD   
111049    26429663  10/04/2013 12:12:07 AM  10/04/2013 01:03:44 AM   NYPD   
111050    26437763  10/04/2013 12:11:00 AM                     NaN    DEP   
111051    26432955  10/04/2013 12:08:15 AM  10/04/2013 12:48:02 AM   NYPD   
111052    26437035  10/04/2013 12:08:00 AM  10/04/2013 12:13:00 AM    DEP   
111053    26433197  10/04/2013 12:08:00 AM  10/04/2013 12:00:00 PM   DSNY   
111054    26426060  10/04/2013 12:06:39 AM  10/04/2013 12:31:16 AM   NYPD   
111055    26430628  10/04/2013 12:06:28 AM  10/04/2013 12:21:39 AM   NYPD   
111056    26431648  10/04/2013 12:06:26 AM  10/23/2013 08:14:52 AM    DOT   
111057    26437034  10/04/2013 12:06:00 AM                     NaN    DEP   
111058    26426094  10/04/2013 12:05:12 AM  10/04/2013 01:08:29 AM   NYPD   
1110

# 2.2 Selecting columns and rows

To select a column, we index with the name of the column, like this:

In [4]:
complaints['Complaint Type']

0          Noise - Street/Sidewalk
1                  Illegal Parking
2               Noise - Commercial
3                  Noise - Vehicle
4                           Rodent
5               Noise - Commercial
6                 Blocked Driveway
7               Noise - Commercial
8               Noise - Commercial
9               Noise - Commercial
10        Noise - House of Worship
11              Noise - Commercial
12                 Illegal Parking
13                 Noise - Vehicle
14                          Rodent
15        Noise - House of Worship
16         Noise - Street/Sidewalk
17                 Illegal Parking
18          Street Light Condition
19              Noise - Commercial
20        Noise - House of Worship
21              Noise - Commercial
22                 Noise - Vehicle
23              Noise - Commercial
24                Blocked Driveway
25         Noise - Street/Sidewalk
26          Street Light Condition
27            Harboring Bees/Wasps
28         Noise - S

To get the first 5 rows of a dataframe, we can use a slice: `df[:5]`.

This is a great way to get a sense for what kind of information is in the dataframe -- take a minute to look at the contents and get a feel for this dataset.

In [5]:
complaints[:5]

Unique Key            Created Date             Closed Date Agency  \
0    26589651  10/31/2013 02:08:41 AM                     NaN   NYPD   
1    26593698  10/31/2013 02:01:04 AM                     NaN   NYPD   
2    26594139  10/31/2013 02:00:24 AM  10/31/2013 02:40:32 AM   NYPD   
3    26595721  10/31/2013 01:56:23 AM  10/31/2013 02:21:48 AM   NYPD   
4    26590930  10/31/2013 01:53:44 AM                     NaN  DOHMH   

                               Agency Name           Complaint Type  \
0          New York City Police Department  Noise - Street/Sidewalk   
1          New York City Police Department          Illegal Parking   
2          New York City Police Department       Noise - Commercial   
3          New York City Police Department          Noise - Vehicle   
4  Department of Health and Mental Hygiene                   Rodent   

                     Descriptor        Location Type Incident Zip  \
0                  Loud Talking      Street/Sidewalk        11432   
1  Commercial Overnight Parking      Street/Sidewalk        11378   
2              Loud Music/Party  Club/Bar/Restaurant        10032   
3                Car/Truck Horn      Street/Sidewalk        10023   
4  Condition Attracting Rodents           Vacant Lot        10027   

   Incident Address      Street Name   Cross Street 1  \
0  90-03 169 STREET       169 STREET        90 AVENUE   
1         58 AVENUE        58 AVENUE         58 PLACE   
2     4060 BROADWAY         BROADWAY  WEST 171 STREET   
3    WEST 72 STREET   WEST 72 STREET  COLUMBUS AVENUE   
4   WEST 124 STREET  WEST 124 STREET     LENOX AVENUE   

                     Cross Street 2 Intersection Street 1  \
0                         91 AVENUE                   NaN   
1                         59 STREET                   NaN   
2                   WEST 172 STREET                   NaN   
3                  AMSTERDAM AVENUE                   NaN   
4  ADAM CLAYTON POWELL JR BOULEVARD                   NaN   

  Intersection Street 2 Address Type      City Landmark Facility Type  \
0                   NaN      ADDRESS   JAMAICA      NaN      Precinct   
1                   NaN    BLOCKFACE   MASPETH      NaN      Precinct   
2                   NaN      ADDRESS  NEW YORK      NaN      Precinct   
3                   NaN    BLOCKFACE  NEW YORK      NaN      Precinct   
4                   NaN    BLOCKFACE  NEW YORK      NaN           NaN   

     Status                Due Date Resolution Action Updated Date  \
0  Assigned  10/31/2013 10:08:41 AM         10/31/2013 02:35:17 AM   
1      Open  10/31/2013 10:01:04 AM                            NaN   
2    Closed  10/31/2013 10:00:24 AM         10/31/2013 02:39:42 AM   
3    Closed  10/31/2013 09:56:23 AM         10/31/2013 02:21:10 AM   
4   Pending  11/30/2013 01:53:44 AM         10/31/2013 01:59:54 AM   

  Community Board    Borough  X Coordinate (State Plane)  \
0       12 QUEENS     QUEENS                   1042027.0   
1       05 QUEENS     QUEENS                   1009349.0   
2    12 MANHATTAN  MANHATTAN                   1001088.0   
3    07 MANHATTAN  MANHATTAN                    989730.0   
4    10 MANHATTAN  MANHATTAN                    998815.0   

   Y Coordinate (State Plane) Park Facility Name Park Borough  School Name  \
0                    197389.0        Unspecified       QUEENS  Unspecified   
1                    201984.0        Unspecified       QUEENS  Unspecified   
2                    246531.0        Unspecified    MANHATTAN  Unspecified   
3                    222727.0        Unspecified    MANHATTAN  Unspecified   
4                    233545.0        Unspecified    MANHATTAN  Unspecified   

  School Number School Region  School Code School Phone Number School Address  \
0   Unspecified   Unspecified  Unspecified         Unspecified    Unspecified   
1   Unspecified   Unspecified  Unspecified         Unspecified    Unspecified   
2   Unspecified   Unspecified  Unspecified         Unspecified    Unspecified   
3   

We can combine these to get the first 5 rows of a column:

In [6]:
complaints['Complaint Type'][:5]

0    Noise - Street/Sidewalk
1            Illegal Parking
2         Noise - Commercial
3            Noise - Vehicle
4                     Rodent
Name: Complaint Type, dtype: object

and it doesn't matter which direction we do it in:

In [7]:
complaints[:5]['Complaint Type']

0    Noise - Street/Sidewalk
1            Illegal Parking
2         Noise - Commercial
3            Noise - Vehicle
4                     Rodent
Name: Complaint Type, dtype: object

# 2.3 Selecting multiple columns

What if we just want to know the complaint type and the borough, but not the rest of the information? Pandas makes it really easy to select a subset of the columns: just index with list of columns you want.

In [8]:
complaints[['Complaint Type', 'Borough']]

Complaint Type        Borough
0        Noise - Street/Sidewalk         QUEENS
1                Illegal Parking         QUEENS
2             Noise - Commercial      MANHATTAN
3                Noise - Vehicle      MANHATTAN
4                         Rodent      MANHATTAN
5             Noise - Commercial         QUEENS
6               Blocked Driveway         QUEENS
7             Noise - Commercial         QUEENS
8             Noise - Commercial      MANHATTAN
9             Noise - Commercial       BROOKLYN
10      Noise - House of Worship       BROOKLYN
11            Noise - Commercial      MANHATTAN
12               Illegal Parking      MANHATTAN
13               Noise - Vehicle          BRONX
14                        Rodent       BROOKLYN
15      Noise - House of Worship      MANHATTAN
16       Noise - Street/Sidewalk  STATEN ISLAND
17               Illegal Parking       BROOKLYN
18        Street Light Condition       BROOKLYN
19            Noise - Commercial      MANHATTAN
20      Noise - House of Worship       BROOKLYN
21            Noise - Commercial      MANHATTAN
22               Noise - Vehicle         QUEENS
23            Noise - Commercial       BROOKLYN
24              Blocked Driveway  STATEN ISLAND
25       Noise - Street/Sidewalk  STATEN ISLAND
26        Street Light Condition       BROOKLYN
27          Harboring Bees/Wasps      MANHATTAN
28       Noise - Street/Sidewalk      MANHATTAN
29        Street Light Condition  STATEN ISLAND
...                          ...            ...
111039        Noise - Commercial      MANHATTAN
111040        Noise - Commercial      MANHATTAN
111041                     Noise       BROOKLYN
111042   Noise - Street/Sidewalk      MANHATTAN
111043        Noise - Commercial       BROOKLYN
111044   Noise - Street/Sidewalk      MANHATTAN
111045              Water System      MANHATTAN
111046                     Noise       BROOKLYN
111047           Illegal Parking         QUEENS
111048   Noise - Street/Sidewalk      MANHATTAN
111049        Noise - Commercial       BROOKLYN
111050                     Noise      MANHATTAN
111051        Noise - Commercial      MANHATTAN
111052              Water System      MANHATTAN
111053         Derelict Vehicles         QUEENS
111054   Noise - Street/Sidewalk       BROOKLYN
111055        Noise - Commercial       BROOKLYN
111056     Street Sign - Missing         QUEENS
111057                     Noise      MANHATTAN
111058        Noise - Commercial       BROOKLYN
111059   Noise - Street/Sidewalk      MANHATTAN
111060                     Noise      MANHATTAN
111061        Noise - Commercial         QUEENS
111062              Water System      MANHATTAN
111063              Water System      MANHATTAN
111064   Maintenance or Facility       BROOKLYN
111065           Illegal Parking         QUEENS
111066   Noise - Street/Sidewalk      MANHATTAN
111067        Noise - Commercial       BROOKLYN
111068          Blocked Driveway       BROOKLYN

[111069 rows x 2 columns]

That showed us a summary, and then we can look at the first 10 rows:

In [9]:
complaints[['Complaint Type', 'Borough']][:10]

Complaint Type    Borough
0  Noise - Street/Sidewalk     QUEENS
1          Illegal Parking     QUEENS
2       Noise - Commercial  MANHATTAN
3          Noise - Vehicle  MANHATTAN
4                   Rodent  MANHATTAN
5       Noise - Commercial     QUEENS
6         Blocked Driveway     QUEENS
7       Noise - Commercial     QUEENS
8       Noise - Commercial  MANHATTAN
9       Noise - Commercial   BROOKLYN

# 2.4 What's the most common complaint type?

This is a really easy question to answer! There's a `.value_counts()` method that we can use:

In [10]:
complaints['Complaint Type'].value_counts()

HEATING                                 14200
GENERAL CONSTRUCTION                     7471
Street Light Condition                   7117
DOF Literature Request                   5797
PLUMBING                                 5373
PAINT - PLASTER                          5149
Blocked Driveway                         4590
NONCONST                                 3998
Street Condition                         3473
Illegal Parking                          3343
Noise                                    3321
Traffic Signal Condition                 3145
Dirty Conditions                         2653
Water System                             2636
Noise - Commercial                       2578
ELECTRIC                                 2350
Broken Muni Meter                        2070
Noise - Street/Sidewalk                  1928
Sanitation Condition                     1824
Rodent                                   1632
Sewer                                    1627
Taxi Complaint                    

If we just wanted the top 10 most common complaints, we can do this:

In [11]:
complaint_counts = complaints['Complaint Type'].value_counts()
complaint_counts[:10]

HEATING                   14200
GENERAL CONSTRUCTION       7471
Street Light Condition     7117
DOF Literature Request     5797
PLUMBING                   5373
PAINT - PLASTER            5149
Blocked Driveway           4590
NONCONST                   3998
Street Condition           3473
Illegal Parking            3343
Name: Complaint Type, dtype: int64

But it gets better! We can plot them!

In [12]:
complaint_counts[:10].plot(kind='bar')

<style>
    @font-face {
        font-family: "Computer Modern";
        src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');
    }
    div.cell{
        width:800px;
        margin-left:16% !important;
        margin-right:auto;
    }
    h1 {
        font-family: Helvetica, serif;
    }
    h4{
        margin-top:12px;
        margin-bottom: 3px;
       }
    div.text_cell_render{
        font-family: Computer Modern, "Helvetica Neue", Arial, Helvetica, Geneva, sans-serif;
        line-height: 145%;
        font-size: 130%;
        width:800px;
        margin-left:auto;
        margin-right:auto;
    }
    .CodeMirror{
            font-family: "Source Code Pro", source-code-pro,Consolas, monospace;
    }
    .text_cell_render h5 {
        font-weight: 300;
        font-size: 22pt;
        color: #4057A1;
        font-style: italic;
        margin-bottom: .5em;
        margin-top: 0.5em;
        display: block;
    }
    
    .warning{
        color: rgb( 240, 20, 20 )
        }  